In [1]:
import xarray as xr
import sys
import pandas as pd
import numpy as np
import xesmf as xe
#import warnings
import dask
import sys
from math import nan

import matplotlib.pyplot as plt

from CASutils import calendar_utils as cal
dask.config.set(**{'array.slicing.split_large_chunks': False})
#warnings.filterwarnings('ignore')

In [2]:
import dask
dask.config.set({'distributed.dashboard.link': 'https://jupyterhub.ucar.edu/dav/user/{USER}/proxy/{port}/status'})

In [ ]:
# get the workers going
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(
    cores = 1,
    memory = '20GB',
    processes = 1,
    queue = 'casper',
    local_directory='$TMPDIR',
    resource_spec='select=1:ncpus=1:mem=20GB',
    project='P04010022',
    walltime='12:00:00',
    interface='ib0')

# scale up
cluster.scale(24)

# change your urls to the dask dashboard so that you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)


In [4]:
cluster

PBSCluster('tcp://10.12.206.54:41821', workers=0, threads=0, memory=0 B)

In [ ]:
#cluster.close()

In [5]:
# do this until you see you've got some workers
client

Client Scheduler: tcp://10.12.206.54:41821 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/islas/proxy/45545/status,Cluster Workers: 24 Cores: 24 Memory: 480.00 GB


In [6]:
# location of ERA5 data on RDA
filepath="/gpfs/fs1/collections/rda/data/ds633.0/e5.oper.an.pl/"
filepathps="/gpfs/fs1/collections/rda/data/ds633.0/e5.oper.an.sfc/"
# output location
outpath="/glade/scratch/islas/processed/era5/moisturebudget/"

In [7]:
ystart=2014 ; yend=2020 ; nyears=yend-ystart+1

In [8]:
# open up CESM data to get the output grid.
cesmdat = xr.open_dataset("/glade/campaign/cesm/collections/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/monthly/PHIS/f.e11.F1850C5CNTVSST.f09_f09.002.cam.h0.PHIS.040101-050012.nc")
grid_out = xr.Dataset({'lat': (['lat'], cesmdat.lat)}, {'lon': (['lon'], cesmdat.lon)})

In [9]:
%%time
reusewgt=False
wgtfile=outpath+"wgtfile.nc"
for iyear in range(ystart,yend+1,1):
    for imon in range(1,13,1):
        print(str(iyear)+'-'+str(imon))
        monstr=str(imon).zfill(2)
        
        # Read in full column fields
        ds = xr.open_mfdataset(
        filepath + str(iyear)+monstr+"*/*_[u,v,q].*.nc",
        coords="minimal",
        join="override",
        decode_times=True,
        use_cftime=True,
        chunks={"time":24, "level":15, "latitude":721, "longitude":1440},
        parallel=True,
        data_vars = "minimal",
        compat = "override")
        
        ds = ds.loc[{"time":ds.time.dt.hour.isin([0,6,12,18])}]
        
        # moisture fluxes
        uq = ds.U*ds.Q
        uq = uq.rename('uq')
        vq = ds.V*ds.Q
        vq = vq.rename('vq')
        
        ds = xr.merge([ds.Q, uq, vq])
    
        # Read in surface pressure
        dsps = xr.open_mfdataset(
        filepathps + str(iyear)+monstr+"*/*_sp.*.nc",
        coords="minimal",
        join="override",
        decode_times=True,
        use_cftime=True,
        chunks={"time":24, "latitude":721, "longitude":1440},
        parallel=True,
        data_vars="minimal",
        compat="override")
        
        ps = dsps.SP
        ps = ps.rename('ps')
        ps = ps.loc[{"time": ps.time.dt.hour.isin([0,6,12,18])}]
        
        ds = xr.merge([ds, ps])
        
        # regrid onto the CESM grid
        regridder = xe.Regridder(ds.Q, grid_out, 'bilinear', periodic=True,
                                reuse_weights=reusewgt, filename=wgtfile)
        regridded = regridder(ds).persist()             
                      
    
        timeout = regridded.time.mean('time')
        regridded = regridded.mean('time', keepdims=True)
        regridded = regridded.assign_coords(time=[pd.to_datetime(monstr+"-"+str(iyear))])
    
        regridded.to_netcdf(outpath+'Qfluxes_'+str(iyear)+'-'+monstr+'.nc')
        
        client.cancel(regridded)

2014-1


/glade/u/home/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/dask/array/core.py:377: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  o = func(*args, **kwargs)
/glade/u/home/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/xesmf/frontend.py:508: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs,


2014-2


/glade/u/home/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/dask/array/core.py:377: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  o = func(*args, **kwargs)
/glade/u/home/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/xesmf/frontend.py:508: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs,


2014-3


...

2020-11


/glade/u/home/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/dask/array/core.py:377: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  o = func(*args, **kwargs)
/glade/u/home/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/xesmf/frontend.py:508: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs,


2020-12


/glade/u/home/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/dask/array/core.py:377: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  o = func(*args, **kwargs)
/glade/u/home/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/xesmf/frontend.py:508: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs,


CPU times: user 47min 33s, sys: 13min 46s, total: 1h 1min 19s
Wall time: 7h 20min 5s
